In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne.preprocessing import ICA
from scipy import signal
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def preprocess_eeg(vhdr_file):
    """
    Preprocess EEG data with comprehensive artifact removal.
    
    This function combines the robust preprocessing pipeline from Method 1 with
    the extra processing steps from Method 2:
      - Additional artifact checks (ECG artifact removal)
      - Channel selection for ICA (only EEG and EOG channels)
      - Explicit reference projection

    Parameters
    ----------
    vhdr_file : str
        Path to BrainVision header file.
    save_dir : str or None
        Directory to save preprocessed data plots (optional).

    Returns
    -------
    raw : mne.io.Raw
        Preprocessed EEG data with artifacts removed.
    """
    # 1. Load the raw data
    raw = mne.io.read_raw_brainvision(vhdr_file, preload=True)    
    # 3. Drop unnecessary channels if they exist
    if 'Mass' in raw.ch_names:
        raw.drop_channels('Mass')
    
    # 4. Set non-EEG channel types if available
    non_eeg_channels = {
        'VPVA': 'eog',
        'VNVB': 'eog',
        'HPHL': 'eog',
        'HNHR': 'eog',
        'Erbs': 'ecg',
        'OrbOcc': 'emg',
    }
    channel_types = {ch: typ for ch, typ in non_eeg_channels.items() if ch in raw.ch_names}
    if channel_types:
        raw.set_channel_types(channel_types)
    
    # 5. Set the montage (use on_missing='ignore' to avoid errors for auxiliary channels)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage, on_missing='ignore')
    
    # 6. Apply a notch filter (50 Hz) to EEG channels and bandpass filter (0.1 - 40 Hz)
    raw.notch_filter(freqs=50, picks='eeg')
    raw.filter(l_freq=0.1, h_freq=40.0)
    
    # 7. Set average reference with projection and apply the projection
    raw.set_eeg_reference('average', projection=True)
    raw = raw.apply_proj()
    
    # 8. Instantiate the ICA model (with additional iterations for convergence)
    ica = ICA(n_components=20, random_state=42, max_iter=800)
    
    # 9. Pick only EEG and EOG channels for ICA fitting to reduce computational load
    eeg_picks = mne.pick_types(raw.info, eeg=True, eog=True, exclude='bads')
    ica.fit(raw, picks=eeg_picks)
    
    # --- Extra Processing Steps ---
    # ECG artifact removal: detect and exclude ECG-related ICA components.
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw, ch_name='Erbs')
    ica.exclude = ecg_indices  # Start exclusion list with ECG-related components
    
    # EOG artifact removal: detect and add EOG-related ICA components to the exclusion list.
    eog_indices, eog_scores = ica.find_bads_eog(raw, ch_name=['VPVA', 'VNVB', 'HPHL', 'HNHR'])
    ica.exclude.extend(eog_indices)

    # 10. Apply ICA to remove the identified artifact components (modifies raw in place)
    raw = ica.apply(raw)
    # print(f"Preprocessing complete.")
    return raw


In [35]:
import numpy as np
import pandas as pd
import mne
from scipy import signal
from scipy.stats import skew, kurtosis
import os

# Load the participant metadata
participants_df = pd.read_csv("../../../Srihari/iBrains/Temp/TDBRAIN_participants_V2_Req.tsv", sep="\t")

def get_gender(subject_id):
    """
    Get gender from the participants TSV file.
    Returns: 1 (Male), 0 (Female), -1 (Unknown)
    """
    row = participants_df.loc[participants_df["participants_ID"] == subject_id]
    if not row.empty:
        return int(row["gender"].values[0])  # Convert to int (0 or 1)
    return -1  # Default if not found

def extract_features(vhdr_path, gender, condition):
    """
    Extract features from preprocessed EEG data and save to CSV
    
    Parameters:
    -----------
    raw : mne.io.Raw
        Preprocessed EEG data
    condition : str
        Condition of the EEG recording (EO or EC)
    save_path : str, optional
        Path to save the extracted features CSV file
        
    Returns:
    --------
    features_df : pd.DataFrame
        DataFrame containing extracted features
    """
    # Get data and sampling frequency
    raw = preprocess_eeg(vhdr_path)
    # raw.pick_types(eeg=True)  
    other_channels = ['VPVA', 'VNVB', 'HPHL', 'HNHR', 'Erbs', 'OrbOcc']
    raw.pick('eeg')  # Pick only EEG channels
    try:
        raw.drop_channels(other_channels)  # Drop non-EEG channels
    except ValueError:
        print(f"Channels {other_channels} not found in raw data.")
    # raw.drop_channels('Mass')  # Drop 'Mass' channel if it exists
    data = raw.get_data()
    sfreq = raw.info['sfreq']
    ch_names = raw.ch_names
    # print(f"Channel names: {ch_names}")
    # Initialize feature dictionary
    features = {}
    features['gender'] = gender
    
    # Define frequency bands
    bands = {
        'delta': (0.5, 4),
        'theta': (4, 8),
        'alpha': (8, 13),
        'beta': (13, 30),
        'gamma': (30, 40)
    }
    
    # Calculate features for each channel
    for i, ch in enumerate(ch_names):
        # Get channel data
        ch_data = data[i]
        prefix = f"{condition}_{ch.lower()}"
        print(prefix)
        # Time domain features
        features[f'{prefix}_mean'] = np.mean(ch_data)
        features[f'{prefix}_std'] = np.std(ch_data)
        features[f'{prefix}_skew'] = skew(ch_data)
        features[f'{prefix}_kurtosis'] = kurtosis(ch_data)
        
        # Frequency domain features
        freqs, psd = signal.welch(ch_data, fs=sfreq, nperseg=int(sfreq*2))
        features[f'{prefix}_psd_mean'] = np.mean(psd)
        # Band-specific FFT features
        fft_vals = np.abs(np.fft.fft(ch_data))
        fft_freqs = np.fft.fftfreq(len(ch_data), d=1/sfreq)
        
        for band, (fmin, fmax) in bands.items():
            idx_band = np.logical_and(fft_freqs >= fmin, fft_freqs <= fmax)
            band_fft_vals = fft_vals[idx_band]
            features[f'{prefix}_{band}_fft_avg_power'] = np.mean(band_fft_vals)
        
        # Band-specific Morlet Wavelet Transform (MWT)
        valid_freqs = [f for f in freqs if f > 0]  # Ensure only positive frequencies
        
        if valid_freqs:
            mwt = mne.time_frequency.tfr_array_morlet(
                data[np.newaxis, [i], :], sfreq, freqs=valid_freqs, n_cycles=2, output='power'
            ).squeeze()
            
            for j, f in enumerate(valid_freqs):
                for band, (fmin, fmax) in bands.items():
                    if fmin <= f <= fmax:
                        features[f'{prefix}_{band}_mwt_avg_power'] = np.mean(mwt[j])
    
    # Convert to DataFrame and save as CSV
    features_df = pd.DataFrame([features])
    
    return features_df


In [36]:
def process_and_combine(eo_file_path, ec_file_path, output_file,gender):
    all_features = []
    eo=False
    ec=False
    # Process EO file
    try:       
        features_eo = extract_features(eo_file_path,gender,"EO")
        all_features.append(features_eo)
        eo=True
    except Exception as e:
        print(f"Error loading file: {e}")
        return None
    
    features_ec = extract_features(ec_file_path,gender,"EC")
    all_features.append(features_ec)
    ec=True
    
    # Combine EO and EC features
    if eo and ec:
        combined_features = pd.concat(all_features,axis=1)
    # print("*****************************",combined_features.shape,"***********************************")
    # out_path = (out_dir,output_file)
    # Save combined features to a single CSV file
        combined_features.to_csv(output_file,index=False)
        print(f"Features successfully saved to {output_file}")
    # return combined_features

In [37]:
process_and_combine("../dataset_s/healthy/sub-87974621/ses-1/eeg/sub-87974621_ses-1_task-restEC_eeg.vhdr","../dataset_s/healthy/sub-87974621/ses-1/eeg/sub-87974621_ses-1_task-restEO_eeg.vhdr","preprocessed.csv",0.0)

Extracting parameters from ../dataset_s/healthy/sub-87974621/ses-1/eeg/sub-87974621_ses-1_task-restEC_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 60319  =      0.000 ...   120.638 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 3301 samples (6.602 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.8s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 170 (average pulse 84.54907161803713 / min.)
Not setting metadata
170 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 170 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

...

Selecting by number: 20 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.7s.
Using threshold: 0.23 for CTPS ECG detection
Using channel Erbs to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5000 samples (10.000 s)

Number of ECG events detected : 173 (average pulse 86.08821138885664 / min.)
Not setting metadata
173 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 173 events and 501 original

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5000 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (20 components)
    Zeroing out 1 ICA component
    Projecting back using 30 PCA components
Channels ['VPVA', 'VNVB', 'HPHL', 'HNHR', 'Erbs', 'OrbOcc'] not found in raw data.
EC_fp1
EC_fp2
EC_f7
EC_f3
EC_fz
EC_f4
EC_f8
EC_fc3
EC_fcz
EC_fc4
EC_t7
EC_c3
EC_cz
EC_c4
EC_t8
EC_cp3
EC_cpz
EC_cp4
EC_p7
EC_p3
EC_pz
EC_p4
EC_p8
EC_o1
EC_oz
EC_o2
Features successfully saved to preprocessed.csv


In [ ]:
import os

def process_folder(source_folder, destination_folder="features"):
    """
    Processes all subject EEG files in the source folder and saves the combined features to CSV files.

    Args:
        source_folder (str): Path to the folder containing subject EEG data.
        destination_folder (str): Path to save the processed CSV files.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get all subject directories
    subject_dirs = [d for d in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, d))]

    for subject_id in subject_dirs:
        eeg_path = os.path.join(source_folder, subject_id, "ses-1", "eeg")
        ec_path = os.path.join(eeg_path, f"{subject_id}_ses-1_task-restEC_eeg.vhdr")
        eo_path = os.path.join(eeg_path, f"{subject_id}_ses-1_task-restEO_eeg.vhdr")
        output_path = os.path.join(destination_folder, f"{subject_id}_restcombined_eeg.csv")
        gender=get_gender(subject_id)
        if os.path.exists(ec_path) and os.path.exists(eo_path):
            print(f"Processing {ec_path} and {eo_path}...")
            process_and_combine(ec_path, eo_path, output_path,gender)
        else:
            print(f"Warning: Missing files for {subject_id}")

In [ ]:
# Example usage
process_folder("../dataset_s/healthy")